In [1]:
# **1. Mengimpor Library dan Menyiapkan Dataset**
# Library utama untuk pengolahan data dan PyTorch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# **Memuat dataset**
file_path = '/content/heart.csv' # Sesuaikan path jika tidak menggunakan Colab
data = pd.read_csv(file_path)

In [3]:
# Memisahkan fitur dan target
X = data.drop('target', axis=1).values  # Fitur
y = data['target'].values               # Target (binary classification)

In [4]:
# Standarisasi data fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:

# Mengubah data ke dalam format PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print("Dimensi data latih:", X_train_tensor.shape)
print("Dimensi data uji:", X_test_tensor.shape)

Dimensi data latih: torch.Size([820, 13])
Dimensi data uji: torch.Size([205, 13])


In [7]:
# **2. Definisi Model MLP**
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_function):
        super(MLPModel, self).__init__()
        layers = []
        in_features = input_size

        # Membuat hidden layers
        for hidden_neurons in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_neurons))
            layers.append(activation_function)  # Fungsi aktivasi
            in_features = hidden_neurons

        # Layer output
        layers.append(nn.Linear(in_features, 2))  # Output 2 kelas (binary classification)
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [8]:
# **3. Fungsi Pelatihan**
def train_model(model, optimizer, criterion, train_loader, test_loader, epochs):
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Validasi
        model.eval()
        with torch.no_grad():
            test_loss = 0
            correct = 0
            total = 0
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                test_loss += criterion(outputs, y_batch).item()
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [9]:
# **4. Parameter Eksperimen**
hidden_layer_configs = [[4], [8], [16, 8], [32, 16, 8]]  # Jumlah neuron di hidden layers
activation_functions = {
    "ReLU": nn.ReLU(),
    "Sigmoid": nn.Sigmoid(),
    "Tanh": nn.Tanh(),
    "Softmax": nn.Softmax(dim=1),
    "Linear": nn.Identity(),
}
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]
epoch_counts = [1, 10, 25, 50, 100, 250]

In [10]:
# **5. Eksperimen: Looping berbagai parameter**
for hidden_layers in hidden_layer_configs:
    for act_name, activation in activation_functions.items():
        for lr in learning_rates:
            for batch_size in batch_sizes:
                # Membuat data loader
                train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
                test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)
                train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

                # Inisialisasi model
                model = MLPModel(input_size=X_train.shape[1], hidden_layers=hidden_layers, activation_function=activation)
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                print(f"\nTraining: Hidden Layers={hidden_layers}, Activation={act_name}, LR={lr}, Batch Size={batch_size}")
                train_model(model, optimizer, criterion, train_loader, test_loader, epochs=25)  # Menggunakan epoch default

Output streaming akan dipotong hingga 5000 baris terakhir.
Epoch 21/25, Loss: 0.6280, Test Loss: 5.7705, Accuracy: 78.54%
Epoch 22/25, Loss: 0.5483, Test Loss: 5.7436, Accuracy: 78.54%
Epoch 23/25, Loss: 0.3948, Test Loss: 5.7266, Accuracy: 78.54%
Epoch 24/25, Loss: 0.2909, Test Loss: 5.7114, Accuracy: 78.54%
Epoch 25/25, Loss: 0.5178, Test Loss: 5.7036, Accuracy: 78.54%

Training: Hidden Layers=[16, 8], Activation=Linear, LR=0.0001, Batch Size=32
Epoch 1/25, Loss: 0.7987, Test Loss: 5.4950, Accuracy: 36.59%
Epoch 2/25, Loss: 0.7251, Test Loss: 5.3962, Accuracy: 37.07%
Epoch 3/25, Loss: 0.6676, Test Loss: 5.3055, Accuracy: 38.54%
Epoch 4/25, Loss: 0.7059, Test Loss: 5.2226, Accuracy: 40.98%
Epoch 5/25, Loss: 0.7035, Test Loss: 5.1437, Accuracy: 44.88%
Epoch 6/25, Loss: 0.7322, Test Loss: 5.0706, Accuracy: 44.88%
Epoch 7/25, Loss: 0.7164, Test Loss: 5.0001, Accuracy: 45.37%
Epoch 8/25, Loss: 0.6650, Test Loss: 4.9322, Accuracy: 45.37%
Epoch 9/25, Loss: 0.6532, Test Loss: 4.8666, Accurac